# Save to numpy data

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Import neccessary libraries

In [7]:
import os
import sys
import glob
import cv2

import pandas as pd
import numpy as np
from natsort import natsorted
import shutil

# import ipyvolume as ipv

# import matplotlib
# from scipy.interpolate import griddata
# matplotlib.use('TkAgg')
# from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
# from matplotlib import cm
# import matplotlib.colors as colors
# from matplotlib.colors import Normalize

# %matplotlib notebook 

import plotly
print(plotly.__version__)

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff


3.7.0


# Global setups

In [8]:
# Global variables
vth_dir = 'data/VB_0519/VTH/'
z_dir = 'data/VB_0519/Z/'
h_dir = 'data/VB_0519/H/'

dimx = 465 + 2
dimy = 324 + 2
dimt = 1

# vth_name = 'vth'+'_'+str(dimt).zfill(4) \
#                 +'_'+str(dimy).zfill(4) \
#                 +'_'+str(dimx).zfill(4)+'.npy'
# z_name   =   'z'+'_'+str(dimt).zfill(4) \
#                 +'_'+str(dimy).zfill(4) \
#                 +'_'+str(dimx).zfill(4)+'.npy'
# h_name   =   'z'+'_'+str(dimt).zfill(4) \
#                 +'_'+str(dimy).zfill(4) \
#                 +'_'+str(dimx).zfill(4)+'.npy'
# print(vth_name)
# print(z_name)
# print(h_name)

vth_files = natsorted(glob.glob(os.path.join(vth_dir, '*.dat')))
z_files = natsorted(glob.glob(os.path.join(z_dir, '*.grd')))
h_files = natsorted(glob.glob(os.path.join(h_dir, '*.grd')))

debug = False
 

# Parse *.dat and *.grd files

In [9]:
######################################################################
def parse_dat(file_name, verbose=True):
    if verbose: print(file_name)
    dat_str = np.genfromtxt(file_name, delimiter = '\t', dtype='unicode')
#     if verbose: 
#         print(dat_str[0])
#         print(dat_str[0][0], dat_str[0][1], dat_str[0][2], dat_str[0][3], dat_str[0][4])
        
    dat = []
    for row in dat_str:
        num = list(map(float, row))
        dat.append(num)
#         if verbose and row[3] != '0': 
#             print(row)
#             print(row[0], row[1], row[2], row[3], row[4])
    if verbose: 
        print(row)
        print(num)
    dat = np.array(dat, dtype=np.float32)
    if verbose: print(dat.shape)
    return dat

######################################################################
def parse_grd(file_name, verbose=True):
    grd_str = np.genfromtxt(file_name, delimiter = ' ', dtype='unicode')
    if verbose: print(grd_str.shape)
    if verbose: print(grd_str[:5])
    grd = []
    for row in grd_str[5:]: # Parse from the line number 5th
        grd.append(np.float32(row.split()))
    grd = np.array(grd, dtype=np.float32)
    if verbose: print(grd.shape)
    return grd

# Drawing with plotly here

In [66]:
######################################################################
# fig = plt.figure(figsize=(10, 10))
# ax = fig.gca(projection='3d')


shutil.rmtree('result', ignore_errors=True)
os.makedirs('result')
###################################################################### 
sample=100
for t in range(sample,sample+1):
    ######################################################################
    # Read the data
    vth_data = parse_dat(vth_files[t], verbose=True).astype(np.float32)
    

    z_data = parse_grd(z_files[t], verbose=debug).astype(np.float32)
    h_data = parse_grd(h_files[0], verbose=debug).astype(np.float32)
    
    # Resize the z and h_data
    z_data = cv2.resize(z_data, (dimx, dimy))
    h_data = cv2.resize(h_data, (dimx, dimy))
    
    x_data, y_data = np.meshgrid(range(dimx), range(dimy))
    
    
    ######################################################################
    # Calculate the magnitudes and the phases
    m_data = np.zeros_like(z_data)
    d_data = np.zeros_like(z_data)
    
    for k in range(vth_data.shape[0]):
        m_data[vth_data[k,2].astype(np.int32), 
               vth_data[k,1].astype(np.int32)] = vth_data[k,3]
        d_data[vth_data[k,2].astype(np.int32), 
               vth_data[k,1].astype(np.int32)] = vth_data[k,4]

    # Calculate the u and v components
    u_data = m_data*np.cos(d_data * np.pi / 180.)
    v_data = m_data*np.sin(d_data * np.pi / 180.)    
    b_data = np.zeros_like(d_data) # background or 0
    
    ######################################################################
    

# cmap = [‘Blackbody’,
# ‘Bluered’,
# ‘Blues’,
# ‘Earth’,
# ‘Electric’,
# ‘Greens’,
# ‘Greys’,
# ‘Hot’,
# ‘Jet’,
# ‘Picnic’,
# ‘Portland’,
# ‘Rainbow’,
# ‘RdBu’,
# ‘Reds’,
# ‘Viridis’,
# ‘YlGnBu’,
# ‘YlOrRd’]    
    
data = [
    go.Surface(
        z=-h_data, 
        colorscale='Jet',
        showscale=False, 
        opacity=0.5,
        contours=go.surface.Contours(
            z=go.surface.contours.Z(
              show=True,
              usecolormap=True,
              project=dict(z=True)
            )
        )
    ),
    
    go.Surface(
        z=z_data, 
        colorscale='Portland',
        showscale=False, 
        opacity=0.8,
        contours=go.surface.Contours(
            z=go.surface.contours.Z(
              show=True,
              usecolormap=True,
              project=dict(z=True)
            )
        )
    ),
    
]
layout = go.Layout(
    title='Water level',
    autosize=False,
    scene=dict(
        camera=dict(eye=dict(x=0, y=0, z=3)),
#         xaxis = dict(
#             range = [0,dimx],),
#         yaxis = dict(
#             range = [0,dimy],),
#         zaxis = dict(
#             range = [-1,1],),
    ),
    width=800,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50
    ),
    
)


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Renderer', showscale=False, colorscale='Greens')


data/VB_0519/VTH/VTH 101h.dat
['363600' '465' '324' '0' '0']
[363600.0, 465.0, 324.0, 0.0, 0.0]
(149872, 5)


In [46]:
# import plotly.plotly as py
# import plotly.graph_objs as go

# import pandas as pd

# # Read data from a csv
# z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')

# data = [
#     go.Surface(
#         z=z_data.values
#     )
# ]
# layout = go.Layout(
#     title='Mt Bruno Elevation',
#     autosize=False,
#     width=500,
#     height=500,
#     margin=dict(
#         l=65,
#         r=50,
#         b=65,
#         t=90
#     )
# )
# fig = go.Figure(data=data, layout=layout)
# py.iplot(fig, filename='elevations-3d-surface')

In [47]:
# z_data.values.dtype